In [14]:
def to_button(s, length):
    digits = [int(c) for c in s[1:-1].split(",")]
    buttons = [0] * length
    for d in digits:
        buttons[d] = 1
    return buttons


def parse_input_line(input_str):
    pattern, *groups = input_str.split(" ")
    solution = [c == '#' for c in pattern[1:-1]]
    buttons = [to_button(b, len(solution)) for b in groups[:-1]]

    return solution, buttons

def parse_inputs(lines):
    return [parse_input_line(line) for line in lines]

def calculate_combinations(length):
    n = 2 ** length
    combos = [int_to_binary(i, length) for i in range(1, n)]
    return sorted(combos, key=lambda x: x.count('1'))

def int_to_binary(i, length):
    binary_str = bin(i)[2:]  # Remove '0b' prefix
    binary_str = binary_str.zfill(length)  # Pad with zeros to length
    return [bit == '1' for bit in binary_str]


In [15]:

def xor_array(arr1, arr2):
    return [a != b for a, b in zip(arr1, arr2)]

def solve(target, buttons):
    combinations = calculate_combinations(len(buttons))
    sorted_combinations = sorted(combinations, key=lambda x: x.count(True))

    for combo in sorted_combinations:
        solved, solution = check_combination(target, combo, buttons)
    
        if solved:
            return solution

def check_combination(target, combination, buttons):
        button_matrix = [b for i, b in enumerate(buttons) if combination[i]]

        result = [False] * len(target)

        for b in button_matrix:
            result = xor_array(result, b)   
            
        if result == target:
            return True, combination

        return False, None

In [2]:
inputs = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}""".splitlines()


In [16]:
with open("inputs/day10.txt", "r") as f:
    inputs = f.read().splitlines()

In [17]:

input_data = parse_inputs(inputs)

solutions = [solve(solution, buttons) for solution, buttons in input_data]
row_solutions = [r.count(True) for r in solutions]
print(f"Button presses per row:  {row_solutions}")
print(f"Solution: {sum(row_solutions)}")

Button presses per row:  [1, 4, 5, 2, 4, 2, 1, 3, 3, 2, 6, 2, 1, 1, 1, 2, 1, 2, 4, 2, 5, 4, 1, 3, 2, 1, 2, 2, 5, 4, 4, 1, 1, 3, 4, 1, 1, 3, 1, 2, 1, 2, 3, 1, 1, 2, 4, 2, 2, 1, 1, 4, 2, 2, 4, 4, 5, 2, 5, 2, 4, 1, 1, 1, 5, 2, 2, 1, 5, 1, 3, 1, 1, 5, 2, 6, 2, 2, 5, 2, 5, 2, 2, 1, 3, 3, 3, 4, 2, 5, 2, 2, 3, 1, 1, 5, 1, 3, 1, 6, 2, 4, 2, 3, 5, 1, 1, 6, 3, 6, 3, 2, 3, 6, 2, 4, 3, 1, 2, 1, 2, 5, 1, 5, 6, 1, 3, 6, 4, 6, 4, 3, 1, 1, 2, 3, 3, 1, 3, 1, 2, 3, 1, 4, 1, 2, 3, 2, 4, 1, 1]
Solution: 401


In [7]:
# Part 2
def parse_jolts(input_str):
    *nonjolts, jolts = input_str.split(" ")
    return [int(x) for x in jolts[1:-1].split(",")]

input_data = parse_inputs(inputs)

jolts = [parse_jolts(line) for line in inputs]


def solve_part_2(jolt, buttons):

    total_jolt = sum(jolt)
    button_sizes = [sum(b) for b in buttons]

    viable_presses = generate_viable_presses(button_sizes, total_jolt,   jolt, [], buttons)
    viable_presses_padded = [p + [0]*(len(button_sizes) - len(p)) for p in viable_presses]
    viable_presses_sorted = sorted(viable_presses_padded, key=lambda x: sum(x))

    print(f"Total viable presses for jolt {jolt}: {len(viable_presses_sorted)}")

    for presses in viable_presses_sorted:
        if is_valid_press(presses, buttons, jolt):
            return sum(presses)


def is_valid_press(presses, buttons, target):
    for i, tgt in enumerate(target):
        actual = sum(b[i] * p for b, p in zip(buttons, presses))
        if actual != tgt:
            return False
    
    return True


def generate_viable_presses(button_sizes, target_jolt, target, proceeding, buttons):

    jolts = 0
    presses = 0

    other_presses = []

    for idx, tgt in enumerate(target):
        actual = sum(b[idx] * presses for b in buttons)
        if actual > tgt:
            return []

    while(jolts < target_jolt):        
        if jolts < target_jolt and len(button_sizes) > 1:
            others = generate_viable_presses(button_sizes[1:], target_jolt - jolts, target, proceeding + [presses], buttons)
            other_presses.extend([[presses] + o for o in others])
        
        jolts += button_sizes[0]
        presses += 1

    if jolts == target_jolt:
        return [[presses]] + other_presses

    return other_presses


values = []
for i, j in enumerate(jolts):
    print(f"Solving Row {i}")
    buttons = input_data[i][1]
    buttons.sort(key=lambda x: sum(x), reverse=True)
    value = solve_part_2(j, buttons)
    values.append(value)

print(f"Part 2 solutions: {sum(values)}")

Solving Row 0
Total viable presses for jolt [29, 29, 11, 11, 40]: 7106
Solving Row 1


KeyboardInterrupt: 

In [ ]:
import numpy as np
from scipy.optimize import linprog


def parse_jolts(input_str):
    *_, jolts = input_str.split(" ")
    return [int(x) for x in jolts[1:-1].split(",")]

def parse_buttons(input_str):
    _, *groups, _ = input_str.strip().split(" ")
    b = [g[1:-1] for g in groups]
    b_split = [[int(n) for n in g.split(",")] for g in b]
    return b_split

def solve_with_linprog(target, button_matrix):
    """
    Solve: find coefficients x such that button_matrix @ x = target
    Minimize: sum(x) (total button presses)
    """
    # Convert to numpy arrays
    A_eq = np.array(button_matrix).T  # Transpose so each row is a constraint
    b_eq = np.array(target)
    
    # Objective: minimize sum of button presses
    c = np.ones(len(button_matrix))
    
    # Bounds: non-negative integer variables
    bounds = [(0, None) for _ in range(len(button_matrix))]
    
    # Solve linear program
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    
    if result.success:
        # Round to nearest integers and verify
        x_int = np.round(result.x).astype(int)
        
        # Verify the integer solution works
        if np.allclose(A_eq @ x_int, b_eq):
            return x_int, sum(x_int)
        else:
            print("Rounding didn't produce valid integer solution")
            return None, None
    else:
        print("No solution found")
        return None, None

jolts = [parse_jolts(line) for line in inputs]
buttons = [parse_buttons(line) for line in inputs]


presses = []
for i, j in enumerate(jolts):
    b = buttons[0]

    bz = []
    for bx in b:
        zeros = [0] * len(j)
        for by in bx:
            zeros[by] = 1
        bz.append(zeros)

    solution, total_presses = solve_with_linprog(j, bz)
    print(f"Row {i} solution: {solution}, total presses: {total_presses}")
    presses.append(total_presses)

print(presses)





Row 0 solution: [1 2 0 4 0 3], total presses: 10
No solution found
Row 1 solution: None, total presses: None
No solution found
Row 2 solution: None, total presses: None


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'